# Core Functions

> This page defines all boilerplate functions for analyzing and evaluating the NLP tools in the 2K paper.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from dotenv import load_dotenv

## Wikidata Queries


The functions interact with Wikidata by sending SPARQL queries to the Wikidata Query Service (WDQS), and the goal is to retrieve information about items or properties based on specific search criteria (e.g., matching a search term in labels, descriptions, or aliases).

In [ ]:
#| export
import aiohttp
import asyncio
import pandas as pd
import nest_asyncio

async def _fetch_query(session: str, # The aiohttp client session.
                        query: str  # The SPARQL query string.
                        ) -> str:   # The JSON response from the Wikidata SPARQL endpoint.
    
    """
    Asynchronously fetch the query result.
    """
    url = 'https://query.wikidata.org/sparql'
    headers = {'User-Agent': 'Mozilla/5.0', 'Accept': 'application/sparql-results+json'}
    async with session.get(url, headers=headers, params={'query': query, 'format': 'json'}) as response:
        if response.status == 200:
            return await response.json()
        return None

async def _search_wikidata_properties_async(search_term: str    # The term to search for in property names and aliases.
                                            ) -> pd.DataFrame:  # A DataFrame of found properties with their ID, label, description, and aliases, sorted by Property ID.
    """
    Asynchronously search Wikidata properties by name or aliases and return a sorted pandas DataFrame.
    """
    query = f'''
    SELECT DISTINCT ?property ?propertyLabel ?propertyDescription (GROUP_CONCAT(DISTINCT ?alias;separator=", ") AS ?aliases)
    WHERE {{
      ?property a wikibase:Property .
      {{?property rdfs:label ?label FILTER (LANG(?label) = "en").}}
      OPTIONAL {{?property skos:altLabel ?alias FILTER (LANG(?alias) = "en").}}
      FILTER(CONTAINS(LCASE(?label), LCASE("{search_term}")) || CONTAINS(LCASE(?alias), LCASE("{search_term}"))).
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY ?property ?propertyLabel ?propertyDescription
    LIMIT 5
    '''

    async with aiohttp.ClientSession() as session:
        json_response = await _fetch_query(session, query)
        if json_response:
            results = json_response.get('results', {}).get('bindings', [])
            data = [{
                "Property ID": result['property']['value'].split('/')[-1][1:],
                "Label": result['propertyLabel']['value'],
                "Description": result['propertyDescription']['value'],
                "Aliases": result.get('aliases', {}).get('value', 'N/A')
            } for result in results]
            
            df = pd.DataFrame(data)
            df['Property ID'] = pd.to_numeric(df['Property ID'], errors='coerce')
            df = df.sort_values(by='Property ID').reset_index(drop=True)
            return df
        else:
            print("Failed to retrieve data")
            return pd.DataFrame()

# Running the asynchronous function
def search_wikidata_properties(search_term: str # A string containing the term to search for in the property names and aliases.
                            ) -> pd.DataFrame:  # A DataFrame of found properties with their ID, label, description, and aliases, sorted by Property ID.
    """
    Main function to asynchronously search Wikidata properties by name or aliases and return a sorted pandas DataFrame.
    """
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(_search_wikidata_properties_async(search_term))


Example of using `search wikidata properties` function, where the search term is "**used for**".

In [ ]:
df_props = search_wikidata_properties("used for")
df_props

## Utils

In [ ]:
#| export
import pandas as pd

def excel_to_csv_to_df(excel_path, csv_path):
    """
    Reads an Excel file, writes its contents to a CSV file, and then reads the CSV
    file back into a DataFrame.
    
    Parameters:
    - excel_path: str, path to the source Excel file.
    - csv_path: str, path where the CSV file will be saved.
    
    Returns:
    - df: pandas.DataFrame, the content of the CSV file as a DataFrame.
    """
    # Read and store content of an excel file
    read_file = pd.read_excel(excel_path)
    
    # Write the dataframe object into csv file
    read_file.to_csv(csv_path, index=None, header=True)
    
    # Read csv file and convert into a dataframe object
    df = pd.read_csv(csv_path)
    
    # Return the dataframe
    return df



Example usage

In [ ]:
df = excel_to_csv_to_df("../evaluations/qualitative/unirel_eval.xlsx", "../evaluations/qualitative/unirel_eval.csv")
display(df.head(3))


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()